## Problem Setup
Consider the nonlinear dynamical system
$$\begin{align}
    x_{t+1} {}={} & f(x_t, w_t),
    \\
    y_t {}={}     & h(x_t) + v_t,
  \end{align}$$
where $x_0$, $w_t$ and $v_t$ are temporally and mutually uncorrelated, $w_t$ and $v_t$ follow known distributions (not necessarily normal)
$$\begin{align}
    p_{w_t}(w) {}\propto{}   & \exp [-\ell_w(w)],
    \\
    p_{v_t}(v) {}\propto{}   & \exp [-\ell_v(v)],
    \\
    p_{x_0}(x_0) {}\propto{} & \exp [-\ell_{x_0}(x_0)].
  \end{align}$$
The disturbances can be constrained — we may assume that $w_t\in W$ and $v_t \in V$ (nonempty, closed), Constrained states: $x_t \in X$. The EKF cannot take into account constraints. To describe the constraints we assume that $\ell_w(w) {}={} \infty$, for $w\notin W$, $\ell_v(v) {}={}$ for  $v\notin V$, and $\ell_{x_0}(x_0) {}={} \infty$, for $x_0\notin X$.

## Full Information Estimation
Our objective is to estimate $x_{0:T}=(x_0, \ldots, x_{T})$ and $w_{0:T-1}=(w_0, \ldots, w_{T-1})$
from measurements $y_{0:T-1}=(y_0, \ldots, y_{T-1})$ using a MAP approach which leads to the optimisation problem
$$\begin{align*}
    \operatorname*{Minimise}_{x_{0:T}, w_{0:T-1}}\ & \ell_{x_0}(x_0) + \sum_{t=0}^{T-1}\ell_{w_t}(w_t) + \ell_{v_t}(y_t - h(x_t)),
    \\
    \text{subject to: }              & x_{t+1}=f(x_t, w_t),\, t\in{\rm I\!N}_{[0, T-1]},
    \\
                                     & x_{t}\in X,\, t\in{\rm I\!N}_{[0, T]}.
  \end{align*}$$
  At time $T$, having observed $y_{0:T-1}$ we can solve the above problem to estimate $x_{0:T}$ and $w_{0:T-1}$.

## Application to Chemical Reactor

**See first:** [EKF](./5_ekf)

Consider the following chemical reaction take takes place in gaseous phase
$$
2A {}\longrightarrow{} B,
$$
with rate coefficient $k=0.16$ and reaction rate $r=kP_A^2$, where $P_A$ and $P_B$ are the \textit{partial pressures} of $A$ and $B$ respectively. The state is the vector $x=[P_A ~ P_B]^\intercal$ and the system dynamics is
$$
x_{t+1} =
\underbracket[0.5pt]{\begin{bmatrix}
    \frac{x_{t, 1}}{2k\Delta t x_{t, 1} + 1}
    \\
    x_{t, 2} + \frac{k \Delta t x_{t, 1}^2}{2k\Delta t x_{t, 1} + 1}
  \end{bmatrix}}_{F(x_t)} + w_t,
$$
where $\Delta t = 0.1$ is the sampling time. We can measure the total pressure, that is
$$y_t = \begin{bmatrix}
  1 & 1
\end{bmatrix}x_t + v_t,$$
where $v_t\sim\mathcal{N}(0, 0.1^2)$ ($R=0.1^2$) and $w_t \sim \mathcal{N}(0, 0.001^2 I_2)$ ($Q=0.001^2 I_2$).

Suppose that $x_0 = [3 ~ 1]^\intercal$ while our prior knowledge about $x_0$ is $\bar{x}_0 = [3.1 ~ 1.1]^\intercal$ and $P_0 = 6^2 I_2$.